<a href="https://colab.research.google.com/github/GarethWilhelm/TS-Feature-Clustering/blob/TSV-reader/notebooks/1a.%20data_prep_tsv_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packages

In [2]:
# !pip install -q oec
# import oec
import pandas as pd
import numpy as np
import json
from random import sample
import random
import itertools

import os, os.path, csv, requests, pathlib
from datetime import datetime
import math
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style
import matplotlib as mpl
mpl.style.use('ggplot')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


## OEC data API functions

In [3]:
# Generic data processors
def remove_null_values(df,threshold:int=0.9):
  pct_null = df.isnull().sum() / len(df)
  missing_features = pct_null[pct_null > threshold].index
  df.drop(missing_features, axis=1, inplace=True)
  df.fillna(0,inplace=True)

def check_outliers(df):
    col = list(df)
    outliers = pd.DataFrame(columns=['columns','Outliers'])
    
    for column in col:
        if column in df.select_dtypes(include=np.number).columns:
            q1 = df[column].quantile(0.25) 
            q3 = df[column].quantile(0.75)
            below = q1 - (1.5*q3 - q1)
            above = q3 + (1.5*q3 - q1)
            outliers = outliers.append({'columns':column,'Outliers':df.loc[(df[column] < below) | (df[column] > above)].shape[0]},ignore_index=True)
    return outliers
    
def removing_outliers(dataframe):
    cols = list(dataframe)
    for col in cols:
        if col in dataframe.select_dtypes(include=np.number).columns:
            dataframe[col] = winsorize(dataframe[col], limits=[0.1, 0.1],inclusive=(True, True))
    
    return dataframe

## Generate dataset
- OEC API endpoint down as at 16 June 2020

- This data has been downloaded and saved in the folder indicated
No need to run this if there no new data needed
- [Resource on nicer sns and plt plots](https://www.drawingfromdata.com/setting-figure-size-using-seaborn-and-matplotlib)


In [26]:
PATH = '/content/drive/My Drive/Stellenbosch/Gareth/sitc/year_origin_sitc_rev2.tsv'

def load(country_code,path):
  df = pd.read_csv(f"{path}/year_origin_sitc_rev2.tsv",sep="\t")
  df = df[df.origin==country_code]
  return df
df = load('zaf',PATH)


products = pd.read_csv(f'{PATH}/AllProducts.txt', sep = " ", header=None)
products.columns = ['product_codes']
codes = products.product_codes.unique()
df = df[df['sitc4'].isin(codes)]
df.head()
df.to_csv(f"{PATH}/zaf_exports_df.csv")
products.head()

,product_codes
0,1110
1,1121
2,1122
3,1123
4,1124
